In [ ]:
import requests
import pandas as pd
import re
from sqlalchemy import create_engine, inspect
from dotenv import load_dotenv
import os   
# Load environment variables from .env file
load_dotenv()
# Define the API key



api_key = os.getenv('api_key')

In [ ]:
class PostgresManager:
    def __init__(self, host, port, dbname, user, password):
        self.db_params = {
            'host': host,
            'port': port,
            'dbname': dbname,
            'user': user,
            'password': password
        }
        self.engine_str = (
            f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
        )
        self.engine = create_engine(self.engine_str)
        print("✅ PostgreSQL connection initialized.")

    def upload_dataframe(self, df: pd.DataFrame, table_name: str, if_exists='replace'):
        """
        Uploads a DataFrame to PostgreSQL.
        - if_exists: 'replace', 'append', or 'fail'
        """
        try:
            df.to_sql(table_name, self.engine, if_exists=if_exists, index=False, method='multi')
            print(f"✅ Data uploaded to table '{table_name}'.")
        except Exception as e:
            print(f"❌ Failed to upload to '{table_name}': {e}")

    def get_tickers_from_30min_tables(self):
        """
        Extracts all tickers from tables that match the {ticker}_30MinData format.
        """
        try:
            inspector = inspect(self.engine)
            all_tables = inspector.get_table_names()
            pattern = re.compile(r'^(.*)_30MinData$', re.IGNORECASE)
            tickers = [match.group(1).upper() for table in all_tables if (match := pattern.match(table))]
            return tickers
        except Exception as e:
            print(f"❌ Failed to inspect tables: {e}")
            return []

# ✅ Example Usage:
pg = PostgresManager(
    host=os.getenv('host'),
    port=os.getenv('port'),
    dbname=os.getenv('dbname'),
    user=os.getenv('user'),
    password=os.getenv('password')
)

# Upload example:
# pg.upload_dataframe(your_dataframe, 'YourTableName')

# Get ticker names
tickers = pg.get_tickers_from_30min_tables()
print("Tickers found:", tickers)


✅ PostgreSQL connection initialized.
Tickers found: ['AVGO', 'MA', 'JNJ', 'PM', 'IBM', 'NOW', 'TBB', 'RTX', 'ADBE', 'SCHW', 'TMO', 'C', 'GILD', 'PANW', 'CMCSA', 'MU', 'APH', 'BX', 'ICE', 'WELL', 'AMT', 'BMYMP', 'SOJC', 'CEG', 'HCA', 'DUKB', 'IBKR', 'ELV', 'INTC', 'AJG', 'TDG', 'CVS', 'RSG', 'ORLY', 'MMM', 'DELL', 'SCCO', 'APO', 'ZTS', 'ECL', 'SNPS', 'RCL', 'BRK-B', 'NFLX', 'HD', 'TMUS', 'WFC', 'AXP', 'ISRG', 'VZ', 'TXN', 'SPGI', 'BSX', 'TJX', 'RCIT', 'PFE', 'COF', 'CRWD', 'LMT', 'COP', 'MSFT', 'BRK-A', 'ORCL', 'BAC', 'UNH', 'CVX', 'NVDA', 'TSLA', 'COST', 'ABBV', 'GE', 'ABT', 'MS', 'MRK', 'BKNG', 'CAT', 'BA', 'NEE', 'APP', 'AMAT', 'ADP', 'LOW', 'VRTX', 'LRCX', 'KKR', 'MSTR', 'PLD', 'CME', 'SOJE', 'SOJD', 'NKE', 'FI', 'SHW', 'EQIX', 'ABNB', 'PH', 'CI', 'CDNS', 'FTNT', 'AAPL', 'WMT', 'XOM', 'KO', 'CRM', 'MCD', 'DIS', 'GS', 'UBER', 'QCOM', 'AMGN', 'HON', 'DE', 'UNP', 'GEV', 'ANET', 'MMC', 'ADI', 'KLAC', 'SBUX', 'MO', 'BMY', 'SO', 'WM', 'DASH', 'CTAS', 'DUK', 'MCK', 'MCO', 'MDLZ', 'UPS', 'A

In [4]:


def get_company_profile(ticker, api_key):
    """Fetches company profile and returns a dictionary of relevant metrics."""
    url = f"https://financialmodelingprep.com/stable/profile?symbol={ticker}&apikey={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            data = response.json()
            if data:
                profile = data[0]
                profile['symbol'] = ticker
                return profile
        except Exception as e:
            print(f"⚠️ Error parsing data for {ticker}: {e}")
    else:
        print(f"⚠️ Failed to fetch profile for {ticker}. Status code: {response.status_code}")
    
    return None

def build_company_profile_table(tickers, api_key):
    """Returns a DataFrame where rows are companies and columns are profile metrics."""
    profiles = []

    for ticker in tickers:
        print(f"📥 Fetching profile for {ticker}")
        profile = get_company_profile(ticker, api_key)
        if profile:
            profiles.append(profile)

    if profiles:
        df = pd.DataFrame(profiles)
        df.set_index('companyName', inplace=True, drop=True)
        return df
    else:
        print("❌ No profile data collected.")
        return pd.DataFrame()

# Example usage
company_profiles_df = build_company_profile_table(tickers, api_key)

print(company_profiles_df.head())


📥 Fetching profile for AVGO
📥 Fetching profile for MA
📥 Fetching profile for JNJ
📥 Fetching profile for PM
📥 Fetching profile for IBM
📥 Fetching profile for NOW
📥 Fetching profile for TBB
📥 Fetching profile for RTX
📥 Fetching profile for ADBE
📥 Fetching profile for SCHW
📥 Fetching profile for TMO
📥 Fetching profile for C
📥 Fetching profile for GILD
📥 Fetching profile for PANW
📥 Fetching profile for CMCSA
📥 Fetching profile for MU
📥 Fetching profile for APH
📥 Fetching profile for BX
📥 Fetching profile for ICE
📥 Fetching profile for WELL
📥 Fetching profile for AMT
📥 Fetching profile for BMYMP
📥 Fetching profile for SOJC
📥 Fetching profile for CEG
📥 Fetching profile for HCA
📥 Fetching profile for DUKB
📥 Fetching profile for IBKR
📥 Fetching profile for ELV
📥 Fetching profile for INTC
📥 Fetching profile for AJG
📥 Fetching profile for TDG
📥 Fetching profile for CVS
📥 Fetching profile for RSG
📥 Fetching profile for ORLY
📥 Fetching profile for MMM
📥 Fetching profile for DELL
📥 Fetching profile

In [6]:
pg.upload_dataframe(company_profiles_df, 'CompanyProfiles', if_exists='replace')

✅ Data uploaded to table 'CompanyProfiles'.


In [7]:
def fetch_and_store_fundamentals(tickers, api_key, pg_manager):
    def get_quarterly_data(url_base, ticker, period):
        params = {
            'symbol': ticker,
            'period': period,
            'limit': 20,
            'apikey': api_key,
        }
        full_url = f"{url_base}?{'&'.join([f'{key}={value}' for key, value in params.items()])}"
        response = requests.get(full_url)
        if response.status_code == 200:
            try:
                df = pd.DataFrame(response.json())
                if not df.empty:
                    df['date'] = pd.to_datetime(df['date'])
                    df['period'] = period
                return df
            except ValueError:
                print(f"⚠️ JSON decoding failed for {url_base} - {period}")
                return pd.DataFrame()
        else:
            print(f"⚠️ Request failed for {url_base} - {period}")
            return pd.DataFrame()

    def get_all_fundamental_data(ticker):
        income_url = "https://financialmodelingprep.com/stable/income-statement"
        balance_url = "https://financialmodelingprep.com/stable/balance-sheet-statement"
        cashflow_url = "https://financialmodelingprep.com/stable/cash-flow-statement"

        all_merged = []

        for period in ['Q1', 'Q2', 'Q3', 'Q4']:
            income_df = get_quarterly_data(income_url, ticker, period)
            balance_df = get_quarterly_data(balance_url, ticker, period)
            cashflow_df = get_quarterly_data(cashflow_url, ticker, period)

            if not (income_df.empty or balance_df.empty or cashflow_df.empty):
                for df in [income_df, balance_df, cashflow_df]:
                    df.set_index('date', inplace=True)

                merged = income_df.join(balance_df, how='inner', lsuffix='_income', rsuffix='_balance')
                merged = merged.join(cashflow_df, how='inner', rsuffix='_cashflow')
                merged.reset_index(inplace=True)
                merged['period'] = period
                all_merged.append(merged)

        if all_merged:
            final_df = pd.concat(all_merged, ignore_index=True)
            final_df.sort_values(by='date', inplace=True)
            return final_df
        else:
            print(f"❌ No combined data for {ticker}")
            return pd.DataFrame()

    # Loop through tickers and push data
    for ticker in tickers:
        print(f"🔍 Fetching fundamentals for {ticker}...")
        df = get_all_fundamental_data(ticker)
        if not df.empty:
            table_name = f"{ticker}_FundamentalsData"
            try:
                pg_manager.upload_dataframe(df, table_name, if_exists='replace')
            except Exception as e:
                print(f"❌ Failed to upload data for {ticker}: {e}")
        else:
            print(f"⚠️ Skipping upload for {ticker} due to empty DataFrame.")


In [8]:

fetch_and_store_fundamentals(tickers, api_key, pg)

🔍 Fetching fundamentals for AVGO...
✅ Data uploaded to table 'AVGO_FundamentalsData'.
🔍 Fetching fundamentals for MA...
✅ Data uploaded to table 'MA_FundamentalsData'.
🔍 Fetching fundamentals for JNJ...
✅ Data uploaded to table 'JNJ_FundamentalsData'.
🔍 Fetching fundamentals for PM...
✅ Data uploaded to table 'PM_FundamentalsData'.
🔍 Fetching fundamentals for IBM...
✅ Data uploaded to table 'IBM_FundamentalsData'.
🔍 Fetching fundamentals for NOW...
✅ Data uploaded to table 'NOW_FundamentalsData'.
🔍 Fetching fundamentals for TBB...
✅ Data uploaded to table 'TBB_FundamentalsData'.
🔍 Fetching fundamentals for RTX...
✅ Data uploaded to table 'RTX_FundamentalsData'.
🔍 Fetching fundamentals for ADBE...
✅ Data uploaded to table 'ADBE_FundamentalsData'.
🔍 Fetching fundamentals for SCHW...
✅ Data uploaded to table 'SCHW_FundamentalsData'.
🔍 Fetching fundamentals for TMO...
✅ Data uploaded to table 'TMO_FundamentalsData'.
🔍 Fetching fundamentals for C...
✅ Data uploaded to table 'C_FundamentalsD

/tmp/ipykernel_2038/1196150514.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_merged, ignore_index=True)


✅ Data uploaded to table 'ORLY_FundamentalsData'.
🔍 Fetching fundamentals for MMM...
✅ Data uploaded to table 'MMM_FundamentalsData'.
🔍 Fetching fundamentals for DELL...
✅ Data uploaded to table 'DELL_FundamentalsData'.
🔍 Fetching fundamentals for SCCO...
✅ Data uploaded to table 'SCCO_FundamentalsData'.
🔍 Fetching fundamentals for APO...
✅ Data uploaded to table 'APO_FundamentalsData'.
🔍 Fetching fundamentals for ZTS...
✅ Data uploaded to table 'ZTS_FundamentalsData'.
🔍 Fetching fundamentals for ECL...
✅ Data uploaded to table 'ECL_FundamentalsData'.
🔍 Fetching fundamentals for SNPS...
✅ Data uploaded to table 'SNPS_FundamentalsData'.
🔍 Fetching fundamentals for RCL...
✅ Data uploaded to table 'RCL_FundamentalsData'.
🔍 Fetching fundamentals for BRK-B...
✅ Data uploaded to table 'BRK-B_FundamentalsData'.
🔍 Fetching fundamentals for NFLX...
✅ Data uploaded to table 'NFLX_FundamentalsData'.
🔍 Fetching fundamentals for HD...
✅ Data uploaded to table 'HD_FundamentalsData'.
🔍 Fetching funda